In [1]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

## Analyzing Potential Correlations between Countries with widespread Purtussis Vaccination policies and cases of COVID-19

Using the WHO world coverage data for vaccinations from: https://www.who.int/immunization/monitoring_surveillance/data/en/

I obtained covid-19 cases by country from: https://ourworldindata.org/coronavirus-country-by-country.




In [0]:
root = '/content/drive/My Drive/STEM Fellowship - Undergrad BDC 2020/Datasets/'
Vaccine_Coverage = pd.read_csv(root + 'WHO - Vaccine World Coverage.csv')

In [25]:
print(Vaccine_Coverage.ISO_code.nunique())

194


In [26]:
Vaccine_Coverage

WHO_REGION ISO_code        Cname  ...  Year Percent_covrage  Asterisc
0            EMR      AFG  Afghanistan  ...  1980            33.0       NaN
1            EMR      AFG  Afghanistan  ...  1981             8.0       NaN
2            EMR      AFG  Afghanistan  ...  1982            10.0       NaN
3            EMR      AFG  Afghanistan  ...  1984            11.0       NaN
4            EMR      AFG  Afghanistan  ...  1985            17.0       NaN
...          ...      ...          ...  ...   ...             ...       ...
53778        AFR      ZWE     Zimbabwe  ...  2004            10.0       NaN
53779        AFR      ZWE     Zimbabwe  ...  2005            50.0       NaN
53780        AFR      ZWE     Zimbabwe  ...  2006            50.0       NaN
53781        AFR      ZWE     Zimbabwe  ...  2007            50.0       NaN
53782        AFR      ZWE     Zimbabwe  ...  2010            90.0       NaN

[53783 rows x 8 columns]

### COVID Dataset

In [15]:
#Import covid dataset
root1 = '/content/drive/My Drive/STEM Fellowship - Undergrad BDC 2020/Datasets/'

covid = pd.read_csv(root1 + 'BCG_COVID/owid-covid-data.csv')
covid.head()

iso_code location  ... handwashing_facilities  hospital_beds_per_100k
0      ABW    Aruba  ...                    NaN                     NaN
1      ABW    Aruba  ...                    NaN                     NaN
2      ABW    Aruba  ...                    NaN                     NaN
3      ABW    Aruba  ...                    NaN                     NaN
4      ABW    Aruba  ...                    NaN                     NaN

[5 rows x 29 columns]

Looking at the data we have a plethora of useful information. For this analysis I am interested in total cases and total deaths and the ratio between them. Of use is also the aged_65_older and aged_70_older as well. Also note that we want the most up to date stats, so take the latest date.

In [16]:
#Check the latest available date
latest_date = covid.date.max()
print('Latest Date Available: %s' % (latest_date))

#Check how many locations are available for that date
tot_loc = covid.location.unique()
latest_loc = covid[covid.date == latest_date].location.unique()
print('Total Unique Locations: %i\nLatest Unique Locations: %i' %(len(tot_loc), len(latest_loc)))

#Which locations are not available
locs_not_available = [loc for loc in tot_loc if loc not in latest_loc]
print('Locations not Available for %s: %s'%(latest_date, locs_not_available))

#Unforunately, we want the latest information on spain, so check when that is
spain_latest = covid[covid.location == 'Spain'].date.max()
print('Using %s' %(spain_latest))

covid = covid[covid.date == spain_latest]
covid.head()

Latest Date Available: 2020-05-17
Total Unique Locations: 212
Latest Unique Locations: 209
Locations not Available for 2020-05-17: ['Spain', 'Hong Kong', 'International']
Using 2020-05-16


iso_code     location  ... handwashing_facilities  hospital_beds_per_100k
54       ABW        Aruba  ...                    NaN                     NaN
183      AFG  Afghanistan  ...                 37.746                    0.50
240      AGO       Angola  ...                 26.664                     NaN
292      AIA     Anguilla  ...                    NaN                     NaN
362      ALB      Albania  ...                    NaN                    2.89

[5 rows x 29 columns]

In [17]:
#Filter out columns
covid.index = covid.location #change index to location
cols = ['total_cases','total_deaths','aged_65_older','aged_70_older']
covid = covid.loc[:,cols]
covid.head()

total_cases  total_deaths  aged_65_older  aged_70_older
location                                                            
Aruba                101             3         13.085          7.452
Afghanistan         6402           168          2.581          1.337
Angola                48             2          2.405          1.362
Anguilla               3             0            NaN            NaN
Albania              916            31         13.188          8.643

In [18]:
#Out of curiosity, what are the top 10 countries with populations aged over 70
covid.sort_values('aged_70_older', ascending=False).head(10)

total_cases  total_deaths  aged_65_older  aged_70_older
location                                                          
Japan            16237           725         27.049         18.493
Italy           223885         31610         23.021         16.240
Germany         173772          7881         21.453         15.957
Portugal         28583          1190         21.502         14.924
Greece            2810           160         20.396         14.524
Latvia             970            19         19.754         14.136
Spain           230698         27563         19.436         13.799
Lithuania         1523            54         19.002         13.778
Austria          16068           628         19.202         13.748
Estonia           1766            63         19.452         13.491

### DTP Dataset

I have taken only the countries from the comprehensive vaccine data set (which includes coverage for multiple data sets) with some form of DTP coverage

In [59]:
#Import Date
dtp = Vaccine_Coverage[Vaccine_Coverage.Vaccine == 'DTP3']
#dtp
#grouped_dtp = dtp.groupby(['ISO_code'], sort = False)['Year'].max()
#grouped_dtp

idx = dtp.groupby(['ISO_code'])['Year'].transform(max) == dtp['Year']
latest_dtp = dtp[idx]
latest_dtp.nsmallest(10,'Percent_covrage')

Vaccine_Coverage[Vaccine_Coverage.ISO_code == 'ITA']

WHO_REGION ISO_code  Cname  ...  Year Percent_covrage  Asterisc
23442        EUR      ITA  Italy  ...  2018            91.0       NaN
23443        EUR      ITA  Italy  ...  2018            70.0       NaN
23444        EUR      ITA  Italy  ...  1990            80.0       NaN
23445        EUR      ITA  Italy  ...  1991            80.0       NaN
23446        EUR      ITA  Italy  ...  1992            80.0       NaN
...          ...      ...    ...  ...   ...             ...       ...
23629        EUR      ITA  Italy  ...  2016            11.0       NaN
23630        EUR      ITA  Italy  ...  2017            14.0       NaN
23631        EUR      ITA  Italy  ...  2018            19.0       NaN
23632        EUR      ITA  Italy  ...  2018            91.0       NaN
23633        EUR      ITA  Italy  ...  2018            70.0       NaN

[192 rows x 8 columns]

# Yea this doesnt really help us since all countries in the list have whooping cough vaccinations and the majority have 90% or above coverage within the nation. Only a few nations have between 40-70% coverage and they are not large or developed nations that would be of use. So this data on whooping cough may not be useful at all...

In [0]:
#I wanted to see if this led anywhere so i didnt waste time taking data for every country. So filter out countries from the covid dataset that we dont have
locs = bcg.index.unique()
covid = covid[covid.index.isin(locs)]
covid

total_cases  total_deaths  aged_65_older  aged_70_older
location                                                               
Australia              7019            98         15.504         10.129
Belgium               54644          8959         18.571         12.849
Brazil               218223         14817          8.552          5.060
Canada                74602          5562         16.984         10.797
China                 84038          4637         10.641          5.929
Colombia              14216           546          7.646          4.312
Germany              173772          7881         21.453         15.957
Ecuador               31467          2594          7.104          4.458
Spain                230698         27563         19.436         13.799
Finland                6228           293         21.228         13.264
France               141919         27529         19.718         13.079
United Kingdom       236711         33998         18.517         12.527
Ireland               23956          1518         13.928          8.678
Italy                223885         31610         23.021         16.240
Japan                 16237           725         27.049         18.493
South Korea           11037           262         13.914          8.622
Mexico                45032          4767          6.857          4.321
New Zealand            1148            21         15.322          9.720
Portugal              28583          1190         21.502         14.924
Russia               262843          2418         14.178          9.393
United States       1443397         87568         15.413          9.732

In [0]:
#We're interested in total_deaths/total_cases
covid['death_case_ratio'] = covid.total_deaths.values/covid.total_cases.values
covid.head()

total_cases  total_deaths  ...  aged_70_older  death_case_ratio
location                              ...                                 
Australia         7019            98  ...         10.129          0.013962
Belgium          54644          8959  ...         12.849          0.163952
Brazil          218223         14817  ...          5.060          0.067898
Canada           74602          5562  ...         10.797          0.074556
China            84038          4637  ...          5.929          0.055177

[5 rows x 5 columns]

In [0]:
joint = covid.copy()
joint['Current_BCG'] = bcg.Current_BCG
joint

total_cases  total_deaths  ...  death_case_ratio  Current_BCG
location                                   ...                               
Australia              7019            98  ...          0.013962            N
Belgium               54644          8959  ...          0.163952            N
Brazil               218223         14817  ...          0.067898            Y
Canada                74602          5562  ...          0.074556            N
China                 84038          4637  ...          0.055177            Y
Colombia              14216           546  ...          0.038407            Y
Germany              173772          7881  ...          0.045353            N
Ecuador               31467          2594  ...          0.082436            N
Spain                230698         27563  ...          0.119477            N
Finland                6228           293  ...          0.047046            N
France               141919         27529  ...          0.193977            N
United Kingdom       236711         33998  ...          0.143627            N
Ireland               23956          1518  ...          0.063366            Y
Italy                223885         31610  ...          0.141189            N
Japan                 16237           725  ...          0.044651            Y
South Korea           11037           262  ...          0.023738            Y
Mexico                45032          4767  ...          0.105858            Y
New Zealand            1148            21  ...          0.018293            N
Portugal              28583          1190  ...          0.041633            Y
Russia               262843          2418  ...          0.009199            Y
United States       1443397         87568  ...          0.060668            N

[21 rows x 6 columns]

In [0]:
#Get no BCG vs yes BCG averages
yes_bcg_mean = np.mean(joint[joint.Current_BCG == 'Y'].death_case_ratio)
no_bcg_mean = np.mean(joint[joint.Current_BCG == 'N'].death_case_ratio)

print('Mean Death-Case ratio of countries with National BCG Vaccination: %f\nMean Death-Case ratio of countries without National BCG Vaccination: %f'%(yes_bcg_mean,no_bcg_mean))

Mean Death-Case ratio of countries with National BCG Vaccination: 0.049992
Mean Death-Case ratio of countries without National BCG Vaccination: 0.092044


In [0]:
joint.cov()

total_cases  total_deaths  ...  aged_70_older  death_case_ratio
total_cases       9.512924e+10  5.838165e+09  ...   19327.087100        904.411442
total_deaths      5.838165e+09  4.231144e+08  ...   12192.342640        408.655621
aged_65_older     3.221175e+04  1.596841e+04  ...      23.365018          0.053964
aged_70_older     1.932709e+04  1.219234e+04  ...      17.689832          0.047109
death_case_ratio  9.044114e+02  4.086556e+02  ...       0.047109          0.002692

[5 rows x 5 columns]

### BCG Dataset - WHO

In [0]:
#Import dataset
bcg_who = pd.read_csv('/content/drive/My Drive/Colab Notebooks/WHS4_543.csv')
bcg_who.head(15)

Unnamed: 0  ...  BCG immunization coverage among 1-year-olds (%).38
0                            Country  ...                                             1980.0 
1                        Afghanistan  ...                                                NaN 
2                            Albania  ...                                               93.0 
3                            Algeria  ...                                                NaN 
4                             Angola  ...                                                NaN 
5                          Argentina  ...                                               62.0 
6                            Armenia  ...                                                NaN 
7                            Austria  ...                                               90.0 
8                         Azerbaijan  ...                                                NaN 
9                         Bangladesh  ...                                                NaN 
10                           Belarus  ...                                                NaN 
11                            Belize  ...                                               71.0 
12                             Benin  ...                                                NaN 
13                            Bhutan  ...                                               43.0 
14  Bolivia (Plurinational State of)  ...                                               30.0 

[15 rows x 40 columns]

We see we have an interesting dataset, where the first column is country and the top row is date.
We will have to change this to a format we can analyze properly.

In [0]:
#Get first row
new_header=bcg_who.iloc[0]
#Remove first row from dataset
bcg_who = bcg_who[1:]
#Change type of year from float to int
new_header.values[1:] = new_header.values[1:].astype(int)
#Set column headers as first row
bcg_who.columns = new_header

bcg_who = bcg_who.set_index('Country')

In [0]:
bcg_who.head(20)

0                                 2018  2017  2016  ...  1982  1981  1980
Country                                             ...                  
Afghanistan                       78.0  78.0  78.0  ...  10.0   NaN   NaN
Albania                           99.0  99.0  99.0  ...  92.0  93.0  93.0
Algeria                           99.0  99.0  99.0  ...   NaN   NaN   NaN
Angola                            86.0  83.0  49.0  ...   NaN   NaN   NaN
Argentina                         93.0  95.0  92.0  ...  77.0  70.0  62.0
Armenia                           99.0  99.0  99.0  ...   NaN   NaN   NaN
Austria                            NaN   NaN   NaN  ...  90.0  90.0  90.0
Azerbaijan                        97.0  97.0  98.0  ...   NaN   NaN   NaN
Bangladesh                        99.0  99.0  99.0  ...   1.0   1.0   NaN
Belarus                           98.0  98.0  98.0  ...   NaN   NaN   NaN
Belize                            99.0  90.0  94.0  ...  85.0  71.0  71.0
Benin                             89.0  89.0  88.0  ...   NaN   NaN   NaN
Bhutan                            99.0  99.0  99.0  ...  45.0  44.0  43.0
Bolivia (Plurinational State of)  90.0  93.0  96.0  ...  31.0  30.0  30.0
Bosnia and Herzegovina            95.0  97.0  97.0  ...   NaN   NaN   NaN
Botswana                          98.0  98.0  98.0  ...  94.0  93.0  92.0
Brazil                            90.0  90.0  90.0  ...  64.0  62.0  56.0
Brunei Darussalam                 99.0  99.0  99.0  ...  99.0  99.0  99.0
Bulgaria                          96.0  96.0  96.0  ...  99.0  99.0  99.0
Burkina Faso                      98.0  98.0  98.0  ...   NaN   NaN   NaN

[20 rows x 39 columns]

Now it is important to extract when the BCG vaccine stopped for certain countries. I'll do this by taking the first occurence of a non NaN number, since countries that stopped BCG vaccination have NaN values.

In [0]:
#Fill NA with -1
bcg_who = bcg_who.fillna(-1)
bcg_who['Year_Stopped'] = [bcg_who.columns[np.argmax(test[bcg_who.index == country].values>-1)] for country in test.index]

In [0]:
bcg_who[bcg_who.index == 'Spain']

Empty DataFrame
Columns: [2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, Year_Stopped]
Index: []

In [0]:
bcg_who.index.values

array(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina',
       'Armenia', 'Austria', 'Azerbaijan', 'Bangladesh', 'Belarus',
       'Belize', 'Benin', 'Bhutan', 'Bolivia (Plurinational State of)',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Central African Republic',
       'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo',
       'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba',
       'Czechia', "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Ghana',
       'Greece', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana',
       'Haiti', 'Honduras', 'Hungary', 'India', 'Indon

In [0]:
#Import dataset
bcg_who = pd.read_csv('/content/drive/My Drive/Colab Notebooks/WHS4_543_list.csv')
bcg_who.head()

GHO (CODE)                                    GHO (DISPLAY)  ... StdDev Comments
0   WHS4_543  BCG immunization coverage among 1-year-olds (%)  ...    NaN      NaN
1   WHS4_543  BCG immunization coverage among 1-year-olds (%)  ...    NaN      NaN
2   WHS4_543  BCG immunization coverage among 1-year-olds (%)  ...    NaN      NaN
3   WHS4_543  BCG immunization coverage among 1-year-olds (%)  ...    NaN      NaN
4   WHS4_543  BCG immunization coverage among 1-year-olds (%)  ...    NaN      NaN

[5 rows x 25 columns]

In [0]:
bcg_who['COUNTRY (CODE)'].unique()

array(['AFG', 'AGO', 'ARG', 'ARM', 'ALB', 'ARE', 'BDI', 'BEN', 'AZE',
       'BIH', 'BLR', 'BFA', 'BGD', 'BGR', 'BOL', 'BRA', 'BRN', 'BLZ',
       'CHL', 'CHN', 'CIV', 'BTN', 'BWA', 'COD', 'CMR', 'CAF', 'CRI',
       'COG', 'COK', 'COL', 'COM', 'CPV', 'DJI', 'DMA', 'CUB', 'CZE',
       'EGY', 'DOM', 'DZA', 'ECU', 'FIN', 'EST', 'ETH', 'FJI', 'FRA',
       'ERI', 'GHA', 'FSM', 'GAB', 'GEO', 'GTM', 'GIN', 'GMB', 'GRC',
       'GNB', 'GNQ', 'HRV', 'HTI', 'HUN', 'GUY', 'HND', 'IDN', 'IRL',
       'KEN', 'KGZ', 'KHM', 'KIR', 'IRN', 'IRQ', 'KAZ', 'IND', 'JAM',
       'JOR', 'JPN', 'KNA', 'KOR', 'LKA', 'LAO', 'LBY', 'LCA', 'LVA',
       'LSO', 'LTU', 'MAR', 'MDG', 'LBR', 'MCO', 'MDA', 'MDV', 'MEX',
       'MHL', 'MKD', 'MLI', 'MWI', 'NER', 'NGA', 'MNG', 'MOZ', 'MRT',
       'MYS', 'NAM', 'MLT', 'MMR', 'MNE', 'PER', 'NIC', 'NPL', 'NRU',
       'OMN', 'PAK', 'PAN', 'PHL', 'NIU', 'PNG', 'QAT', 'POL', 'PRK',
       'PRT', 'PRY', 'SDN', 'ROU', 'RUS', 'RWA', 'SAU', 'SGP', 'SLB',
       'SLE', 'SEN',

So I noticed that for countries with BCG vaccination cessation before 1980, there simply is no entry, so I'll use the ISO Alpha-3 country codes to add columns for countries that do no appear.

In [0]:
country_codes = pd.read_csv("/content/drive/My Drive/Colab Notebooks/countries_codes_and_coordinates.csv", index_col=0)
country_codes = country_codes[["Alpha-3 code"]]
country_codes["Alpha-3 code"]=country_codes["Alpha-3 code"].str.replace('"','')
country_codes.head()

Alpha-3 code
Country                    
Afghanistan             AFG
Albania                 ALB
Algeria                 DZA
American Samoa          ASM
Andorra                 AND

In [0]:
missing_idx = [code.strip() not in bcg_who['COUNTRY (CODE)'].unique() for code in country_codes["Alpha-3 code"].values]
country_codes[]

In [0]:
missing_idx

[False,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 